<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/gradientPruningTest3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_val_categorical = tf.keras.utils.to_categorical(y_val_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)

In [ ]:
# 모델 정의
model = models.Sequential()

# Conv1 레이어
model.add(layers.Conv1D(filters=64, kernel_size=16, strides=16, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# Pool1 레이어
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# Conv2 레이어
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu'))

# Conv3 레이어
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu'))

# Conv4 레이어
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='relu'))

# Pool2 레이어
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# Flatten 레이어
model.add(layers.Flatten())

# FC1 레이어
model.add(layers.Dense(units=5000, activation='relu'))

# FC2 레이어
model.add(layers.Dense(units=1000, activation='relu'))

# Output 레이어
model.add(layers.Dense(len(categories), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 750, 64)           2112      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 375, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_9 (Conv1D)           (None, 373, 32)           6176      
                                                                 
 conv1d_10 (Conv1D)          (None, 369, 64)           10304     
                                                                 
 conv1d_11 (Conv1D)          (None, 365, 128)          41088     
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 182, 128)          0         
 g1D)                                                 

In [ ]:
# 모델 훈련
history = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 196s 494ms/step - loss: 1.0158 - accuracy: 0.6032 - val_loss: 0.0445 - val_accuracy: 0.9996
Epoch 2/10
394/394 [==============================] - 196s 497ms/step - loss: 0.0738 - accuracy: 0.9786 - val_loss: 0.0055 - val_accuracy: 0.9985
Epoch 3/10
394/394 [==============================] - 199s 505ms/step - loss: 0.0633 - accuracy: 0.9833 - val_loss: 0.0091 - val_accuracy: 0.9974
Epoch 4/10
394/394 [==============================] - 200s 507ms/step - loss: 0.0809 - accuracy: 0.9821 - val_loss: 4.5352e-04 - val_accuracy: 1.0000
Epoch 5/10
394/394 [==============================] - 200s 508ms/step - loss: 0.0225 - accuracy: 0.9940 - val_loss: 0.0027 - val_accuracy: 0.9996
Epoch 6/10
394/394 [==============================] - 200s 507ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 2.1713e-04 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 199s 505ms/step - loss: 0.0124 - accuracy: 0.9963 - val_loss: 

In [ ]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy before pruning: {test_accuracy}')

85/85 [==============================] - 3s 39ms/step - loss: 0.0360 - accuracy: 0.9933
Test accuracy before pruning: 0.9933333396911621


In [ ]:
# 원본 모델 저장
model.save('model_before_pruning.h5')

In [ ]:
def gradient_based_unstructured_pruning(model, X, y, pruning_threshold=0.0009):
    """
    모델의 모든 레이어에 대해 변화도를 계산하고,
    변화도가 pruning_threshold보다 작은 개별 가중치를 0으로 만듭니다.
    비구조적 프루닝.
    """
    # 손실 함수 정의
    loss_fn = tf.keras.losses.CategoricalCrossentropy()

    # 변화도 계산을 위한 테이프
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        loss = loss_fn(y, predictions)

    # 모델의 모든 가중치에 대한 변화도 계산
    grads = tape.gradient(loss, model.trainable_variables)

    # 각 레이어에 대해 변화도 기반 비구조적 프루닝 적용
    grad_index = 0  # 변화도 배열에서 해당 레이어의 인덱스 관리
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            # 레이어의 가중치와 바이어스 가져오기
            weights, biases = layer.get_weights()

            # 레이어의 가중치에 대한 변화도 가져오기
            weight_grad = grads[grad_index]  # i번째 레이어의 가중치에 대한 변화도
            bias_grad = grads[grad_index + 1]  # i번째 레이어의 바이어스에 대한 변화도
            grad_index += 2  # 가중치와 바이어스 둘 다 처리했으므로 2만큼 인덱스 증가

            # 각 가중치 요소별로 변화도의 절댓값 계산
            weight_grad_abs = np.abs(weight_grad)
            bias_grad_abs = np.abs(bias_grad)

            # 가중치와 변화도의 차원을 맞추기 위해 가중치와 동일한 차원 유지
            weights[weight_grad_abs < pruning_threshold] = 0
            biases[bias_grad_abs < pruning_threshold] = 0

            # 프루닝된 가중치와 바이어스를 레이어에 다시 설정
            layer.set_weights([weights, biases])

In [ ]:
# --- 비제로 가중치 계산 함수 정의 ---
def calculate_non_zero_weights(model):
    """
    모델의 레이어별 비제로 가중치와 전체 가중치의 수를 계산하는 함수.
    Conv 레이어 또는 Dense 레이어의 kernel 가중치와 bias를 확인하고 비제로 가중치 수를 세어줌.
    """
    total_weights = 0  # 전체 가중치 수
    non_zero_weights = 0  # 비제로 가중치 수

    for layer in model.layers:
        # 레이어가 kernel 가중치를 가지고 있는지 확인 (Conv 또는 Dense 레이어)
        if hasattr(layer, 'kernel'):
            # 레이어의 가중치와 바이어스 가져오기
            weights, biases = layer.get_weights()

            # 전체 가중치 수 계산 (weights와 biases의 요소 수를 더함)
            total_weights += np.prod(weights.shape) + np.prod(biases.shape)

            # 비제로 가중치 수 계산
            non_zero_weights += np.count_nonzero(weights) + np.count_nonzero(biases)

    return non_zero_weights, total_weights

In [ ]:
# 프루닝 적용 전 비제로 가중치 계산
non_zero_before, total_before = calculate_non_zero_weights(model)
print(f"Before Pruning: Non-zero weights: {non_zero_before}/{total_before}")

Before Pruning: Non-zero weights: 121549668/121549684


In [ ]:
# 비구조적 프루닝 적용
gradient_based_unstructured_pruning(model, X_train[:100], y_train_categorical[:100])

In [ ]:
# 프루닝 후 모델 재훈련
history_pruned = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 207s 526ms/step - loss: 0.4207 - accuracy: 0.8367 - val_loss: 0.0253 - val_accuracy: 0.9978
Epoch 2/10
394/394 [==============================] - 211s 536ms/step - loss: 0.0545 - accuracy: 0.9866 - val_loss: 0.0076 - val_accuracy: 0.9996
Epoch 3/10
394/394 [==============================] - 211s 535ms/step - loss: 0.0375 - accuracy: 0.9870 - val_loss: 0.0255 - val_accuracy: 0.9974
Epoch 4/10
394/394 [==============================] - 210s 533ms/step - loss: 0.0068 - accuracy: 0.9997 - val_loss: 0.1172 - val_accuracy: 0.9567
Epoch 5/10
394/394 [==============================] - 212s 539ms/step - loss: 0.0072 - accuracy: 0.9979 - val_loss: 0.4665 - val_accuracy: 0.8704
Epoch 6/10
394/394 [==============================] - 209s 532ms/step - loss: 0.0082 - accuracy: 0.9980 - val_loss: 0.0187 - val_accuracy: 0.9937
Epoch 7/10
394/394 [==============================] - 208s 528ms/step - loss: 0.0428 - accuracy: 0.9875 - val_loss: 0.0386 -

In [ ]:
# 프루닝 후 모델 평가
test_loss_pruned, test_accuracy_pruned = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy after pruning: {test_accuracy_pruned}')

85/85 [==============================] - 4s 44ms/step - loss: 0.0600 - accuracy: 0.9793
Test accuracy after pruning: 0.9792592525482178


In [ ]:
# 프루닝 후 모델 저장
model.save('model_after_pruning.h5')

# 모델 사이즈 측정
model_size_before = os.path.getsize('model_before_pruning.h5') / (1024 * 1024)
model_size_after = os.path.getsize('model_after_pruning.h5') / (1024 * 1024)
print(f"Model Size before pruning: {model_size_before:.2f} MB")
print(f"Model Size after pruning: {model_size_after:.2f} MB")

Model Size before pruning: 1391.09 MB
Model Size after pruning: 1391.09 MB


In [ ]:
# 프루닝 후 비제로 가중치 계산
non_zero_after, total_after = calculate_non_zero_weights(model)
print(f"After Pruning: Non-zero weights: {non_zero_after}/{total_after}")

After Pruning: Non-zero weights: 5441662/121549684


In [ ]:
# 비제로 가중치 계산 함수 정의
def calculate_non_zero_weights(model):
    """
    모델의 레이어별 비제로 가중치와 전체 가중치의 수를 계산하는 함수.
    Conv 레이어 또는 Dense 레이어의 kernel 가중치와 bias를 확인하고 비제로 가중치 수를 세어줌.
    """
    total_weights = 0  # 전체 가중치 수
    non_zero_weights = 0  # 비제로 가중치 수

    for layer in model.layers:
        # 레이어가 kernel 가중치를 가지고 있는지 확인 (Conv 또는 Dense 레이어)
        if hasattr(layer, 'kernel'):
            # 레이어의 가중치와 바이어스 가져오기
            weights, biases = layer.get_weights()

            # 전체 가중치 수 계산 (weights와 biases의 요소 수를 더함)
            total_weights += np.prod(weights.shape) + np.prod(biases.shape)

            # 비제로 가중치 수 계산
            non_zero_weights += np.count_nonzero(weights) + np.count_nonzero(biases)

    return non_zero_weights, total_weights

In [ ]:
# 프루닝 전후 비제로 가중치 및 전체 가중치 계산
non_zero_weights_before, total_weights_before = calculate_non_zero_weights(model)
non_zero_weights_after, total_weights_after = calculate_non_zero_weights(model)

In [ ]:
# 결과 출력
print(f"Before pruning: Non-zero weights = {non_zero_weights_before}, Total weights = {total_weights_before}")
print(f"After pruning: Non-zero weights = {non_zero_weights_after}, Total weights = {total_weights_after}")
print(f"Final non-zero weight ratio: {non_zero_weights_after / total_weights_after:.4f}")

Before pruning: Non-zero weights = 5441662, Total weights = 121549684
After pruning: Non-zero weights = 5441662, Total weights = 121549684
Final non-zero weight ratio: 0.0448
